In [5]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Define paths
data_dir = 'Solar_Panel_Dataset/'
categories = ['Clean', 'Dusty', 'Bird-Drop', 'Electrical-Damage', 'Physical-Damage', 'Snow-Covered']
img_size = 128

# Load images
data = []
labels = []

for i, category in enumerate(categories):
    folder_path = os.path.join(data_dir, category)
    for img in os.listdir(folder_path):
        try:
            img_array = cv2.imread(os.path.join(folder_path, img))
            resized = cv2.resize(img_array, (img_size, img_size))
            data.append(resized)
            labels.append(i)
        except Exception as e:
            pass

# Convert to array
X = np.array(data) / 255.0
y = to_categorical(labels)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [6]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

# Base model
base_model = MobileNetV2(input_shape=(img_size, img_size, 3), include_top=False, weights='imagenet')
base_model.trainable = False

# Full model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(6, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train
model.fit(X_train, y_train, validation_data=(X_test, y_test),
          epochs=30, class_weight=class_weight_dict, callbacks=[early_stop])


Epoch 1/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 14s 392ms/step - accuracy: 0.2438 - loss: 2.4865 - val_accuracy: 0.5517 - val_loss: 1.2639
Epoch 2/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 268ms/step - accuracy: 0.5610 - loss: 1.1809 - val_accuracy: 0.6820 - val_loss: 1.0150
Epoch 3/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 267ms/step - accuracy: 0.6534 - loss: 0.9309 - val_accuracy: 0.7318 - val_loss: 0.8728
Epoch 4/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 271ms/step - accuracy: 0.7338 - loss: 0.6512 - val_accuracy: 0.7203 - val_loss: 0.8486
Epoch 5/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 268ms/step - accuracy: 0.7368 - loss: 0.6212 - val_accuracy: 0.7395 - val_loss: 0.7639
Epoch 6/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 268ms/step - accuracy: 0.8195 - loss: 0.5030 - val_accuracy: 0.7663 - val_loss: 0.7501
Epoch 7/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 269ms/step - accuracy: 0.8465 - loss: 0.3999 - val_accuracy: 0.7816 - val_loss: 0.7150
Epoch 8/30
19/19 ━━━━━━━━━━━━━━━━━━━━ 5s 270ms/step - accuracy: 0.8728 - loss: 0.3793 - val_accuracy: 0

In [7]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

print(classification_report(y_true, y_pred_classes, target_names=categories))




9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 383ms/step
                   precision    recall  f1-score   support

            Clean       0.75      0.86      0.80        50
            Dusty       0.81      0.67      0.73        63
        Bird-Drop       0.79      0.87      0.83        62
Electrical-Damage       0.86      0.89      0.88        28
  Physical-Damage       0.77      0.59      0.67        17
     Snow-Covered       0.90      0.93      0.92        41

         accuracy                           0.81       261
        macro avg       0.82      0.80      0.80       261
     weighted avg       0.81      0.81      0.81       261

